<a href="https://colab.research.google.com/github/sarahpursley/dsci551-assignment2/blob/main/tasks-code/q8_GROVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#generated emails 
!cd /content && rm -rf dsci551-assignment2 && git clone https://github.com/sarahpursley/dsci551-assignment2.git
import pandas as pd
import json 
import os
import sys

Cloning into 'dsci551-assignment2'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 195 (delta 79), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (195/195), 40.71 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
mal_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_mal.txt', mode = 'r').readlines()
phish_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_phish.txt', mode = 'r').readlines()
recon_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_recon.txt', mode = 'r').readlines()
se_gen = open('dsci551-assignment2/datasets/generated_text/gpt2_gentext_se.txt', mode = 'r').readlines()


#using text generated from TTR algorithm for just the fraud message content
mal_fraud = open('dsci551-assignment2/datasets/generated_text/m_train.txt', mode = 'r').readlines()
se_fraud = open('dsci551-assignment2/datasets/generated_text/s_train.txt', mode = 'r').readlines()
recon_fraud = open('dsci551-assignment2/datasets/generated_text/r_train_.txt', mode = 'r').readlines()
phish_fraud = open('dsci551-assignment2/datasets/generated_text/p_train.txt', mode = 'r').readlines()

In [ ]:
# clean GPU text
generated_malware = [line for line in mal_gen if line.rstrip() != ''and line != '====================\n']
generated_phising = [line for line in phish_gen if line.rstrip() != ''and line != '====================\n']
generated_recon = [line for line in recon_gen if line.rstrip() != ''and line != '====================\n']
generated_se = [line for line in se_gen if line.rstrip() != ''and line != '====================\n']

In [ ]:
message_dict = {}
index = 0
for message in generated_malware:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'malware'}
  index+=1
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'malware'}
  index+=1

for message in generated_phising:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'phishing'}
  index+=1
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'phishing'}
  index+=1

for message in generated_recon:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'reconnaissance'}
  index+=1
  
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'reconnaissance'}
  index+=1

for message in generated_se:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'social_engineering'}
  index+=1
  
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'machine', 'email_type':'social_engineering'}
  index+=1

#real human emails
for message in se_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'social_engineering'}
  index+=1

for message in mal_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'malware'}
  index+=1

for message in recon_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'reconnaissance'}
  index+=1

for message in phish_fraud:
  message_dict['train_emails_'+ str(index)] = {'email': message, 'label': 'human', 'email_type':'phishing'}
  index+=1

#example output
message_dict['train_emails_0']


In [ ]:
print("There are this many 'lines': " + str(index))

There are this many 'lines': 5290


In [ ]:
!cd /content && rm -rf grover && git clone https://github.com/rowanz/grover.git
%cd /content/grover

!pip install cuda
!pip install keras
!pip install -r requirements-gpu.txt
!python download_model.py base

# FOR THE FIRST TIME YOU MUST UNINSTALL AND REINSTALL TENSORFLOW/TENSORFLOW GPU
# only do this for the first runtime
#!pip uninstall -y tensorflow
#!pip install tensorflow-gpu==1.14.0

In [ ]:
import json
! mkdir input_data

all_emails = message_dict

with open("input_data/train_data.jsonl", 'w') as f:
    for item in all_emails.values():
        f.write(json.dumps(item) + "\n")

In [ ]:
!export PYTHONPATH=$(pwd)
! PYTHONPATH=$(pwd) python lm/train.py --config_file lm/configs/base.json --input_file train_data.jsonl --num_train_steps 10000 --output_dir /content/grover

In [ ]:
!export PYTHONPATH=$(pwd)
! python lm/train.py --help

Traceback (most recent call last):
  File "lm/train.py", line 20, in <module>
    from lm.dataloader import input_fn_builder
ModuleNotFoundError: No module named 'lm'


In [ ]:
! mkdir input_data

In [ ]:
!export PYTHONPATH=$(pwd)
! PYTHONPATH=$(pwd) python discrimination/run_discrimination.py --help


For discrimination finetuning (e.g. saying whether or not the generation is human/grover)

flags:

discrimination/run_discrimination.py:
  --[no]adafactor: Whether to run adafactor
    (default: 'false')
  --additional_data: Should we provide additional input data? maybe.
  --batch_size: Batch size used
    (default: '32')
    (an integer)
  --config_file: The config json file corresponding to the pre-trained news
    model. This specifies the model architecture.
    (default: 'configs/base.json')
  --[no]do_train: Whether to run training.
    (default: 'false')
  --gcp_project: [Optional] Project name for the Cloud TPU-enabled project. If
    not specified, we will attempt to automatically detect the GCE project from
    metadata.
  --init_checkpoint: Initial checkpoint (usually from a pre-trained model).
  --input_data: The input data dir. Should contain the .tsv files (or other data
    files) for the task.
  --iterations_per_loop: How many steps to make in each estimator call.
   

In [ ]:
%cd /content/grover
!export PYTHONPATH=$(pwd)
! PYTHONPATH=$(pwd) python discrimination/run_discrimination.py --input_data input_data/ --init_checkpoint checkpoint --config_file lm/configs/base.json --output_dir content/grover --num_train_epochs 10.0 --do_train

/content/grover
EXITING BECAUSE DO_TRAIN IS FALSE AND PATH DOESNT EXIST




```
Someones exampe idk
```



In [ ]:
`!python discrimination/run_discrimination.py --input_data=./path_to_your_input_data_parent_directory/part8_inputs_total.jsonl --output_dir=gs://YOUR_BUCKET_NAME/outputs --config_file=./path_to_lm_folder_parent_directory/lm/configs/base.json --do_train=True